# Regression Analysis: ARDS Intervention Timing and Outcomes

This notebook performs regression analysis on ARDS patients to examine the relationship between intervention timing and clinical outcomes.

## Analysis Plan:
1. **Data Preparation (15 min)**
   - Load final datasets from analysis_dataset directory
   - Create key timing variables (early_prone, early_nmb)
   - Handle missing data
   - Create severity groups using APACHE + PF ratio

2. **Primary Analysis (30 min)**
   - Logistic regression for mortality outcome
   - Linear regression for length of stay
   - Train on eICU, validate on MIMIC-IV

3. **Visualizations (10 min)**
   - Kaplan-Meier curves
   - Box plots for LOS
   - ROC curves
   - Correlation heatmap

## 1. Setup and Import Libraries

In [14]:
!pip install lifelines

In [15]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, roc_curve, classification_report, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
import scipy.stats as stats
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
import warnings
warnings.filterwarnings('ignore')

# Set style for plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Create output directories
import os
os.makedirs('figures', exist_ok=True)
os.makedirs('results', exist_ok=True)

print("Libraries imported successfully!")

Libraries imported successfully!


## 2. Load Final Datasets

In [16]:
# Define data path
DATA_PATH = '/Users/kavenchhikara/Desktop/projects/SCCM/SCCM-Team2/analysis_dataset'

# Load MIMIC final dataset
try:
    mimic_df = pd.read_parquet(f'{DATA_PATH}/mimic_final.parquet')
    print(f"✅ MIMIC dataset loaded: {len(mimic_df):,} records")
    print(f"   Columns: {len(mimic_df.columns)}")
    print(f"   Unique patients: {mimic_df['patient_id'].nunique():,}")
except Exception as e:
    print(f"❌ Error loading MIMIC data: {e}")
    mimic_df = None

# Load eICU final dataset
try:
    eicu_df = pd.read_parquet(f'{DATA_PATH}/eicu_final.parquet')
    print(f"\n✅ eICU dataset loaded: {len(eicu_df):,} records")
    print(f"   Columns: {len(eicu_df.columns)}")
    print(f"   Unique patients: {eicu_df['patient_id'].nunique():,}")
except Exception as e:
    print(f"❌ Error loading eICU data: {e}")
    eicu_df = None

# Display column comparison
if mimic_df is not None and eicu_df is not None:
    mimic_cols = set(mimic_df.columns)
    eicu_cols = set(eicu_df.columns)
    common_cols = mimic_cols & eicu_cols
    
    print(f"\n📊 Dataset Comparison:")
    print(f"   Common columns: {len(common_cols)}")
    print(f"   MIMIC-only: {len(mimic_cols - eicu_cols)}")
    print(f"   eICU-only: {len(eicu_cols - mimic_cols)}")
    
    print(f"\n🔍 Common columns: {sorted(list(common_cols))[:10]}...") # Show first 10

✅ MIMIC dataset loaded: 7,750,657 records
   Columns: 37
   Unique patients: 4,252

✅ eICU dataset loaded: 1,206,142 records
   Columns: 28
   Unique patients: 16,269

📊 Dataset Comparison:
   Common columns: 23
   MIMIC-only: 14
   eICU-only: 5

🔍 Common columns: ['age_at_admission', 'atracurium_dose', 'cisatracurium_dose', 'disposition_category', 'ecmo_flag', 'fio2_set', 'height_cm', 'hospital_id', 'hospitalization_id', 'new_tracheostomy']...


## 3. Data Preparation (15 minutes)

### 3.1 Create Key Timing Variables

In [17]:
def create_timing_variables(df, dataset_name):
    """
    Create key timing variables for ARDS intervention analysis
    
    Parameters:
    - df: DataFrame with time series data
    - dataset_name: String identifier for the dataset
    
    Returns:
    - DataFrame with timing variables added
    """
    if df is None:
        return None
    
    print(f"\n=== Creating Timing Variables for {dataset_name} ===")
    df_processed = df.copy()
    
    # 1. Calculate PF ratio (PaO2/FiO2)
    if 'pao2' in df.columns and 'fio2_set' in df.columns:
        # Convert FiO2 from percentage to fraction if needed
        fio2_fraction = df['fio2_set'].copy()
        fio2_fraction = np.where(fio2_fraction > 1, fio2_fraction / 100, fio2_fraction)
        
        df_processed['pf_ratio'] = df['pao2'] / fio2_fraction
        # Filter unrealistic values
        df_processed.loc[df_processed['pf_ratio'] > 600, 'pf_ratio'] = np.nan
        df_processed.loc[df_processed['pf_ratio'] < 50, 'pf_ratio'] = np.nan
        
        print(f"   PF ratio calculated: {df_processed['pf_ratio'].notna().sum():,} valid values")
    
    # 2. Calculate SF ratio as backup
    if 'spo2' in df.columns and 'fio2_set' in df.columns:
        fio2_fraction = df['fio2_set'].copy()
        fio2_fraction = np.where(fio2_fraction > 1, fio2_fraction / 100, fio2_fraction)
        
        df_processed['sf_ratio'] = df['spo2'] / fio2_fraction
        # Filter unrealistic values
        df_processed.loc[df_processed['sf_ratio'] > 500, 'sf_ratio'] = np.nan
        df_processed.loc[df_processed['sf_ratio'] < 50, 'sf_ratio'] = np.nan
        
        print(f"   SF ratio calculated: {df_processed['sf_ratio'].notna().sum():,} valid values")
    
    # 3. Create early intervention variables (within 24-48h of ARDS onset)
    if 'time_from_ARDS_onset' in df.columns:
        # Early prone positioning (within 24-48h)
        if 'prone_flag' in df.columns:
            early_prone_mask = (
                (df['prone_flag'] == 1) & 
                (df['time_from_ARDS_onset'] >= 0) & 
                (df['time_from_ARDS_onset'] <= 48)  # Within 48 hours
            )
            
            # Get patients who had early proning
            early_prone_patients = df[early_prone_mask]['patient_id'].unique()
            df_processed['early_prone'] = df_processed['patient_id'].isin(early_prone_patients).astype(int)
            
            print(f"   Early prone positioning: {len(early_prone_patients):,} patients ({len(early_prone_patients)/df['patient_id'].nunique()*100:.1f}%)")
        
        # Early neuromuscular blockade (within 24-48h)
        nmb_cols = ['cisatracurium_dose', 'vecuronium_dose', 'rocuronium_dose', 'atracurium_dose', 'pancuronium_dose']
        existing_nmb_cols = [col for col in nmb_cols if col in df.columns]
        
        if existing_nmb_cols:
            # Check for any NMB use within 48h
            nmb_mask = (
                (df[existing_nmb_cols] > 0).any(axis=1) & 
                (df['time_from_ARDS_onset'] >= 0) & 
                (df['time_from_ARDS_onset'] <= 48)
            )
            
            early_nmb_patients = df[nmb_mask]['patient_id'].unique()
            df_processed['early_nmb'] = df_processed['patient_id'].isin(early_nmb_patients).astype(int)
            
            print(f"   Early NMB: {len(early_nmb_patients):,} patients ({len(early_nmb_patients)/df['patient_id'].nunique()*100:.1f}%)")
        elif 'nmb_used' in df.columns:
            # Use existing nmb_used flag
            nmb_mask = (
                (df['nmb_used'] == 1) & 
                (df['time_from_ARDS_onset'] >= 0) & 
                (df['time_from_ARDS_onset'] <= 48)
            )
            
            early_nmb_patients = df[nmb_mask]['patient_id'].unique()
            df_processed['early_nmb'] = df_processed['patient_id'].isin(early_nmb_patients).astype(int)
            
            print(f"   Early NMB: {len(early_nmb_patients):,} patients ({len(early_nmb_patients)/df['patient_id'].nunique()*100:.1f}%)")
    
    return df_processed

# Create timing variables for both datasets
if mimic_df is not None:
    mimic_processed = create_timing_variables(mimic_df, 'MIMIC-IV')

if eicu_df is not None:
    eicu_processed = create_timing_variables(eicu_df, 'eICU-CRD')


=== Creating Timing Variables for MIMIC-IV ===
   PF ratio calculated: 2,376 valid values
   SF ratio calculated: 1,033,586 valid values
   Early prone positioning: 60 patients (1.4%)
   Early NMB: 278 patients (6.5%)

=== Creating Timing Variables for eICU-CRD ===
   PF ratio calculated: 0 valid values
   SF ratio calculated: 155,921 valid values
   Early prone positioning: 733 patients (4.5%)
   Early NMB: 113 patients (0.7%)


### 3.2 Create Patient-Level Analysis Dataset

In [18]:
def create_patient_level_dataset(df, dataset_name):
    """
    Create patient-level dataset for regression analysis
    
    Parameters:
    - df: DataFrame with time series data
    - dataset_name: String identifier
    
    Returns:
    - DataFrame with one row per patient
    """
    if df is None:
        return None
    
    print(f"\n=== Creating Patient-Level Dataset for {dataset_name} ===")
    
    # Define aggregation rules
    agg_rules = {
        # Demographics (first value)
        'age_at_admission': 'first',
        'sex': 'first',
        
        # Outcomes (static values - should be same across all records)
        'mortality': 'first',
        
        # Respiratory parameters (mean of valid values)
        'pf_ratio': lambda x: x.dropna().mean() if len(x.dropna()) > 0 else np.nan,
        'sf_ratio': lambda x: x.dropna().mean() if len(x.dropna()) > 0 else np.nan,
        'peep_set': lambda x: x.dropna().mean() if len(x.dropna()) > 0 else np.nan,
        'fio2_set': lambda x: x.dropna().mean() if len(x.dropna()) > 0 else np.nan,
        
        # Interventions (max - if ever used)
        'early_prone': 'max',
        'early_nmb': 'max',
        'prone_flag': 'max',
        'nmb_used': 'max',
        'new_tracheostomy': 'max',
        
        # Severity scores
        'height_cm': 'first',
        'weight_kg': 'first'
    }
    
    # Add dataset-specific columns
    if 'APACHE' in df.columns:
        agg_rules['APACHE'] = 'first'
    
    if 'icu_los_days' in df.columns:
        agg_rules['icu_los_days'] = 'first'
    elif 'hospital_los_days' in df.columns:
        agg_rules['hospital_los_days'] = 'first'
    
    if 'ventilator_free_days_28' in df.columns:
        agg_rules['ventilator_free_days_28'] = 'first'
    
    # Handle peep column naming differences
    if 'peep' in df.columns and 'peep_set' not in df.columns:
        agg_rules['peep'] = lambda x: x.dropna().mean() if len(x.dropna()) > 0 else np.nan
        del agg_rules['peep_set']
    
    # Filter to only use columns that exist
    available_cols = {k: v for k, v in agg_rules.items() if k in df.columns}
    
    # Aggregate to patient level
    patient_df = df.groupby('patient_id').agg(available_cols).reset_index()
    
    # Add dataset identifier
    patient_df['dataset'] = dataset_name
    
    print(f"   Patient-level dataset created: {len(patient_df):,} patients")
    print(f"   Columns: {list(patient_df.columns)}")
    
    return patient_df

# Create patient-level datasets
if mimic_processed is not None:
    mimic_patients = create_patient_level_dataset(mimic_processed, 'MIMIC-IV')

if eicu_processed is not None:
    eicu_patients = create_patient_level_dataset(eicu_processed, 'eICU-CRD')


=== Creating Patient-Level Dataset for MIMIC-IV ===
   Patient-level dataset created: 4,252 patients
   Columns: ['patient_id', 'age_at_admission', 'sex', 'mortality', 'pf_ratio', 'sf_ratio', 'peep_set', 'fio2_set', 'early_prone', 'early_nmb', 'prone_flag', 'nmb_used', 'new_tracheostomy', 'height_cm', 'weight_kg', 'icu_los_days', 'ventilator_free_days_28', 'dataset']

=== Creating Patient-Level Dataset for eICU-CRD ===
   Patient-level dataset created: 16,269 patients
   Columns: ['patient_id', 'age_at_admission', 'sex', 'pf_ratio', 'sf_ratio', 'fio2_set', 'early_prone', 'early_nmb', 'prone_flag', 'nmb_used', 'new_tracheostomy', 'height_cm', 'weight_kg', 'APACHE', 'peep', 'dataset']


### 3.3 Create Severity Groups and Handle Missing Data

In [19]:
def create_severity_groups_and_clean(df):
    """
    Create severity groups and handle missing data
    
    Parameters:
    - df: Patient-level DataFrame
    
    Returns:
    - Cleaned DataFrame with severity groups
    """
    if df is None:
        return None
    
    df_clean = df.copy()
    
    # 1. Create PF ratio severity groups (Berlin criteria)
    if 'pf_ratio' in df_clean.columns:
        df_clean['ards_severity_pf'] = pd.cut(
            df_clean['pf_ratio'],
            bins=[0, 100, 200, 300, np.inf],
            labels=['Severe', 'Moderate', 'Mild', 'No_ARDS'],
            ordered=True
        )
    
    # 2. Create SF ratio severity groups (approximation)
    if 'sf_ratio' in df_clean.columns:
        df_clean['ards_severity_sf'] = pd.cut(
            df_clean['sf_ratio'],
            bins=[0, 88, 181, 315, np.inf],
            labels=['Severe', 'Moderate', 'Mild', 'No_ARDS'],
            ordered=True
        )
    
    # 4. Handle missing data with simple imputation
    numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
    valid_numeric_cols = [col for col in numeric_cols if df_clean[col].notna().any()]
    categorical_cols = df_clean.select_dtypes(include=['object']).columns

    # Impute numeric variables with median (only on valid cols)
    imputer_numeric = SimpleImputer(strategy='median')
    df_clean[valid_numeric_cols] = imputer_numeric.fit_transform(df_clean[valid_numeric_cols])
    
    # Impute categorical variables with mode
    imputer_categorical = SimpleImputer(strategy='most_frequent')
    df_clean[categorical_cols] = imputer_categorical.fit_transform(df_clean[categorical_cols])
    
    # 5. Create additional derived variables
    # BMI if height and weight available
    if 'height_cm' in df_clean.columns and 'weight_kg' in df_clean.columns:
        df_clean['bmi'] = df_clean['weight_kg'] / (df_clean['height_cm'] / 100) ** 2
        df_clean['obesity'] = (df_clean['bmi'] >= 30).astype(int)
    
    # Gender encoding
    if 'sex' in df_clean.columns:
        df_clean['male'] = (df_clean['sex'].str.upper().isin(['M', 'MALE'])).astype(int)
    
    print(f"\nData cleaning completed:")
    print(f"  Patients: {len(df_clean):,}")
    print(f"  Missing values: {df_clean.isnull().sum().sum()}")
    
    return df_clean

# Clean both datasets
if mimic_patients is not None:
    print("\n=== Cleaning MIMIC Dataset ===")
    mimic_clean = create_severity_groups_and_clean(mimic_patients)
    print(f"MIMIC severity groups:")
    if 'ards_severity_pf' in mimic_clean.columns:
        print(mimic_clean['ards_severity_pf'].value_counts())

if eicu_patients is not None:
    print("\n=== Cleaning eICU Dataset ===")
    eicu_clean = create_severity_groups_and_clean(eicu_patients)
    print(f"eICU severity groups:")
    if 'ards_severity_pf' in eicu_clean.columns:
        print(eicu_clean['ards_severity_pf'].value_counts())
    if 'severity_group' in eicu_clean.columns:
        print(f"Combined severity groups:")
        print(eicu_clean['severity_group'].value_counts())


=== Cleaning MIMIC Dataset ===

Data cleaning completed:
  Patients: 4,252
  Missing values: 4071
MIMIC severity groups:
ards_severity_pf
Moderate    93
Mild        81
No_ARDS     75
Severe      19
Name: count, dtype: int64

=== Cleaning eICU Dataset ===

Data cleaning completed:
  Patients: 16,269
  Missing values: 39849
eICU severity groups:
ards_severity_pf
Severe      0
Moderate    0
Mild        0
No_ARDS     0
Name: count, dtype: int64


### 3.4 Prepare Analysis Variables

In [24]:
eicu_df.dtypes

hospital_id               int64
patient_id              float64
hospitalization_id        int64
recorded_dttm            object
ARDS_onset_time           int64
time_from_ARDS_onset    float64
APACHE                  float64
sex                      object
age_at_admission          int64
ethinicity               object
disposition_category     object
respiratory_device       object
ecmo_flag                 int64
pao2                    float64
fio2_set                float64
lmp_set                 float64
spo2                    float64
peep                    float64
height_cm               float64
weight_kg               float64
nmb_used                float64
cisatracurium_dose      float64
vecuronium_dose         float64
rocuronium_dose         float64
atracurium_dose         float64
pancuronium_dose        float64
prone_flag                int64
new_tracheostomy          int64
dtype: object

In [ ]:
# Define key variables for analysis
key_predictors = [
    'early_prone', 'early_nmb', 'age_at_admission', 'male',
    'pf_ratio', 'sf_ratio'
]

# Define outcomes
outcomes = ['mortality']

# Add LOS if available
if mimic_clean is not None and 'icu_los_days' in mimic_clean.columns:
    outcomes.append('icu_los_days')
if eicu_clean is not None and 'hospital_los_days' in eicu_clean.columns:
    outcomes.append('hospital_los_days')

print(f"Key predictors: {key_predictors}")
print(f"Outcomes: {outcomes}")

# Check data availability
if mimic_clean is not None:
    available_predictors_mimic = [col for col in key_predictors if col in mimic_clean.columns]
    print(f"\nMIMIC available predictors: {available_predictors_mimic}")
    print(f"MIMIC sample characteristics:")
    print(f"  Early prone: {mimic_clean['early_prone'].sum()} ({mimic_clean['early_prone'].mean()*100:.1f}%)")
    print(f"  Early NMB: {mimic_clean['early_nmb'].sum()} ({mimic_clean['early_nmb'].mean()*100:.1f}%)")
    print(f"  Mortality: {mimic_clean['mortality'].sum()} ({mimic_clean['mortality'].mean()*100:.1f}%)")

if eicu_clean is not None:
    available_predictors_eicu = [col for col in key_predictors if col in eicu_clean.columns]
    print(f"\neICU available predictors: {available_predictors_eicu}")
    print(f"eICU sample characteristics:")
    print(f"  Early prone: {eicu_clean['early_prone'].sum()} ({eicu_clean['early_prone'].mean()*100:.1f}%)")
    print(f"  Early NMB: {eicu_clean['early_nmb'].sum()} ({eicu_clean['early_nmb'].mean()*100:.1f}%)")
    print(f"  Mortality: {eicu_clean['mortality'].sum()} ({eicu_clean['mortality'].mean()*100:.1f}%)")

Key predictors: ['early_prone', 'early_nmb', 'age_at_admission', 'male', 'pf_ratio', 'sf_ratio', 'APACHE']
Outcomes: ['mortality', 'icu_los_days']

MIMIC available predictors: ['early_prone', 'early_nmb', 'age_at_admission', 'male', 'pf_ratio', 'sf_ratio']
MIMIC sample characteristics:
  Early prone: 60.0 (1.4%)
  Early NMB: 278.0 (6.5%)
  Mortality: 999.0 (23.5%)

eICU available predictors: ['early_prone', 'early_nmb', 'age_at_admission', 'male', 'pf_ratio', 'sf_ratio', 'APACHE']
eICU sample characteristics:
  Early prone: 733.0 (4.5%)
  Early NMB: 113.0 (0.7%)


KeyError: 'mortality'